In [1]:
import torch
import os
import pandas as pd
from skimage import io, transform, color
from PIL import Image
import torch.nn as nn

from torchvision import models
from torchvision.io import read_image
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import dataset, dataloader, Dataset
from torchvision import transforms, utils

In [ ]:
!mkdir -p /content/drive/MyDrive/data/state

In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip -O /content/drive/MyDrive/data/testSet.zip

In [ ]:
!unzip -q /content/drive/MyDrive/data/testSet.zip -d /content/drive/MyDrive/data/

In [ ]:
!git clone https://github.com/1-1is0/deep-projects

In [ ]:
!cp deep-projects/q2/*.py .

In [ ]:
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
if os.environ.get("USERNAME") == 'amir':
    base_path = "."
else:
    base_path = "/content/drive/MyDrive"
data_path = f"{base_path}/data/tiny-imagenet-200/train/*"
state_path = f"{base_path}/data/state"


In [27]:
from dataloader import load_data, ImageDataset, Rescale, ToTensor, GetAlexConv
path = "/content/drive/MyDrive/data/tiny-imagenet-200/train/*"
train_df, test_df = load_data(path)
train_dataset = ImageDataset(
    train_df,
    transform=transforms.Compose(
        [
            Rescale((227, 227)),
            ToTensor(),
        ]
    ),
)
test_dataset = ImageDataset(
    test_df,
    transform=transforms.Compose(
        [
            Rescale((227, 227)),
            ToTensor(),
        ]
    ),
)


trainloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, shuffle=True, num_workers=2
)
testloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=True, num_workers=2
)

print("train_dataset:", len(train_dataset))
print("test_dataset", len(test_dataset))


In [28]:
from model import AlexFc8
net = AlexFc8().to(device)

print(net)


In [29]:

import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR, ExponentialLR, StepLR

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

scheduler = StepLR(optimizer, step_size=5, gamma=0.3)


In [ ]:
from model import train_model
dataloader = {
    "train": trainloader,
    "val": testloader,
}
dataset_size = {
    "train": len(train_dataset),
    "val": len(test_dataset),
}

res = train_model(data_loader=dataloader, model=net, criterion=criterion, optimizer=optimizer, scheduler=scheduler, device=device, epochs=50, dataset_size=dataset_size)


print(res)


In [ ]:
!cp /content/drive/MyDrive/data/state/state-AlexFc8.pth /kaggle/working/